In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn import datasets

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance

In [22]:
supermarket_df = pd.read_csv('./data/Supermarket Data.csv')
print(supermarket_df.shape)
supermarket_df.head()

(956574, 22)


,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200732,20071005,6,17,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,UM,YF,994107800547472,L,MM,Top Up,Grocery,STORE00001,LS,E02
1,200733,20071010,4,20,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,MM,OF,994107900512001,L,MM,Full Shop,Fresh,STORE00001,LS,E02
2,200741,20071209,1,11,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,MM,YF,994108700468327,L,MM,Full Shop,Grocery,STORE00001,LS,E02
3,200731,20070929,7,17,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,LA,OF,994107700237811,L,LA,Full Shop,Mixed,STORE00001,LS,E02
4,200737,20071110,7,14,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,NaN,NaN,994108300002212,L,MM,Full Shop,Fresh,STORE00001,LS,E02


In [23]:
supermarket_df['SHOP_DATE'] = pd.to_datetime(supermarket_df.SHOP_DATE, format='%Y%m%d')
supermarket_df.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200732,2007-10-05,6,17,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,UM,YF,994107800547472,L,MM,Top Up,Grocery,STORE00001,LS,E02
1,200733,2007-10-10,4,20,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,MM,OF,994107900512001,L,MM,Full Shop,Fresh,STORE00001,LS,E02
2,200741,2007-12-09,1,11,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,MM,YF,994108700468327,L,MM,Full Shop,Grocery,STORE00001,LS,E02
3,200731,2007-09-29,7,17,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,LA,OF,994107700237811,L,LA,Full Shop,Mixed,STORE00001,LS,E02
4,200737,2007-11-10,7,14,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,NaN,NaN,994108300002212,L,MM,Full Shop,Fresh,STORE00001,LS,E02


In [24]:
def check_basket_prince_sensitive(x,type):
    if x['BASKET_PRICE_SENSITIVITY']==type:
        return str(x.BASKET_ID)
    else:
        return None

supermarket_df['BASKET_SEN_LA'] = supermarket_df.apply(lambda x : check_basket_prince_sensitive(x,'LA'),axis = 1)
supermarket_df['BASKET_SEN_MM'] = supermarket_df.apply(lambda x : check_basket_prince_sensitive(x,'MM'),axis = 1)
supermarket_df['BASKET_SEN_UM'] = supermarket_df.apply(lambda x : check_basket_prince_sensitive(x,'UM'),axis = 1)
supermarket_df['BASKET_SEN_XX'] = supermarket_df.apply(lambda x : check_basket_prince_sensitive(x,'XX'),axis = 1)

supermarket_df.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION,BASKET_SEN_LA,BASKET_SEN_MM,BASKET_SEN_UM,BASKET_SEN_XX
0,200732,2007-10-05,6,17,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,MM,Top Up,Grocery,STORE00001,LS,E02,None,994107800547472,None,None
1,200733,2007-10-10,4,20,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,MM,Full Shop,Fresh,STORE00001,LS,E02,None,994107900512001,None,None
2,200741,2007-12-09,1,11,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,MM,Full Shop,Grocery,STORE00001,LS,E02,None,994108700468327,None,None
3,200731,2007-09-29,7,17,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,LA,Full Shop,Mixed,STORE00001,LS,E02,994107700237811,None,None,None
4,200737,2007-11-10,7,14,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,MM,Full Shop,Fresh,STORE00001,LS,E02,None,994108300002212,None,None


In [25]:
def check_customer_prince_sensitive(x,type):
    if x['CUST_PRICE_SENSITIVITY']==type:
        return str(x.BASKET_ID)
    else:
        return None

supermarket_df['CUSTOMER_SEN_LA'] = supermarket_df.apply(lambda x : check_customer_prince_sensitive(x,'LA'),axis = 1)
supermarket_df['CUSTOMER_SEN_MM'] = supermarket_df.apply(lambda x : check_customer_prince_sensitive(x,'MM'),axis = 1)
supermarket_df['CUSTOMER_SEN_UM'] = supermarket_df.apply(lambda x : check_customer_prince_sensitive(x,'UM'),axis = 1)
supermarket_df['CUSTOMER_SEN_XX'] = supermarket_df.apply(lambda x : check_customer_prince_sensitive(x,'XX'),axis = 1)

supermarket_df.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,STORE_FORMAT,STORE_REGION,BASKET_SEN_LA,BASKET_SEN_MM,BASKET_SEN_UM,BASKET_SEN_XX,CUSTOMER_SEN_LA,CUSTOMER_SEN_MM,CUSTOMER_SEN_UM,CUSTOMER_SEN_XX
0,200732,2007-10-05,6,17,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,LS,E02,None,994107800547472,None,None,None,None,994107800547472,None
1,200733,2007-10-10,4,20,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,LS,E02,None,994107900512001,None,None,None,994107900512001,None,None
2,200741,2007-12-09,1,11,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,LS,E02,None,994108700468327,None,None,None,994108700468327,None,None
3,200731,2007-09-29,7,17,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,LS,E02,994107700237811,None,None,None,994107700237811,None,None,None
4,200737,2007-11-10,7,14,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,LS,E02,None,994108300002212,None,None,None,None,None,None


In [26]:
def check_basket_type(x,type):
    if x['BASKET_TYPE']==type:
        return str(x.BASKET_ID)
    else:
        return None

supermarket_df['BASKET_TYPE_SS'] = supermarket_df.apply(lambda x : check_basket_type(x,'Small Shop'),axis = 1)
supermarket_df['BASKET_TYPE_TU'] = supermarket_df.apply(lambda x : check_basket_type(x,'Top Up'),axis = 1)
supermarket_df['BASKET_TYPE_FS'] = supermarket_df.apply(lambda x : check_basket_type(x,'Full Shop'),axis = 1)
supermarket_df['BASKET_TYPE_XX'] = supermarket_df.apply(lambda x : check_basket_type(x,'XX'),axis = 1)

In [27]:
def check_basket_dominant_mission(x,type):
    if x['BASKET_DOMINANT_MISSION']==type:
        return str(x.BASKET_ID)
    else:
        return None

supermarket_df['BASKET_DOMINANT_FR'] = supermarket_df.apply(lambda x : check_basket_dominant_mission(x,'Fresh'),axis = 1)
supermarket_df['BASKET_DOMINANT_GR'] = supermarket_df.apply(lambda x : check_basket_dominant_mission(x,'Grocery'),axis = 1)
supermarket_df['BASKET_DOMINANT_MI'] = supermarket_df.apply(lambda x : check_basket_dominant_mission(x,'Mixed'),axis = 1)
supermarket_df['BASKET_DOMINANT_NF'] = supermarket_df.apply(lambda x : check_basket_dominant_mission(x,'Non Food'),axis = 1)
supermarket_df['BASKET_DOMINANT_XX'] = supermarket_df.apply(lambda x : check_basket_dominant_mission(x,'XX'),axis = 1)

In [ ]:
customer_df = supermarket_df.groupby(['CUST_CODE']).agg(
    {'trans_date': lambda x : (campaign_date - x.max()).days, # last visit
     'basket_id': pd.Series.nunique, # total visit
     'tran_amount': 'sum', # total spend
     'customer_month': 'max'
    }).reset_index() 

In [ ]:
customer_df = supermarket_df.groupby(['CUST_CODE']).agg(
    {'BASKET_ID': pd.Series.nunique, # total visit
     'SPEND': 'sum', # total visit
     'tran_amount': 'sum', # total spend
     'customer_month': 'max'
    }).reset_index() 